 ## 1. Data Collection

In [ ]:
import requests
import pandas as pd
import time

# Configuration
GITHUB_TOKEN = 'github_pat_3434fc434------' # Paste your token
HEADERS = {'Authorization': f'token {GITHUB_TOKEN}'}
MAX_REPOS = 1000  # We want top 1000 repos
MIN_STARS = 50    # Minimum 50 stars requirement

def get_top_repositories():
    repos = []
    page = 1
    per_page = 100  # Max allowed by GitHub API
    
    while len(repos) < MAX_REPOS:
        url = f'https://api.github.com/search/repositories?q=stars:>={MIN_STARS}&sort=stars&order=desc&page={page}&per_page={per_page}'
        response = requests.get(url, headers=HEADERS)
        
        if response.status_code != 200:
            print(f"Error: {response.status_code}")
            break
            
        data = response.json()
        if not data['items']:
            break
            
        repos.extend(data['items'])
        print(f"Fetched {len(repos)}/{MAX_REPOS} repositories")
        
        # Avoid hitting rate limit (30 requests/minute for authenticated users)
        time.sleep(1)
        page += 1
    
    return repos[:MAX_REPOS]

# Fetch and save data
top_repos = get_top_repositories()
repos_df = pd.DataFrame(top_repos)
repos_df.to_csv('github_top_repos.csv', index=False)

Fetched 100/1000 repositories
Fetched 200/1000 repositories
Fetched 300/1000 repositories
Fetched 400/1000 repositories
Fetched 500/1000 repositories
Fetched 600/1000 repositories
Fetched 700/1000 repositories
Fetched 800/1000 repositories
Fetched 900/1000 repositories
Fetched 1000/1000 repositories


## 2. Feature Extraction

In [2]:
def extract_features(repo_data):
    features = {
        'name': repo_data['name'],
        'full_name': repo_data['full_name'],
        'stars': repo_data['stargazers_count'],
        'forks': repo_data['forks_count'],
        'watchers': repo_data['watchers_count'],
        'open_issues': repo_data['open_issues_count'],
        'size': repo_data['size'],
        'has_wiki': int(repo_data['has_wiki']),
        'has_projects': int(repo_data['has_projects']),
        'has_downloads': int(repo_data['has_downloads']),
        'is_fork': int(repo_data['fork']),
        'archived': int(repo_data['archived']),
        'created_at': repo_data['created_at'],
        'updated_at': repo_data['updated_at'],
        'pushed_at': repo_data['pushed_at'],
        'language': repo_data['language'],
        'license': repo_data['license']['key'] if repo_data['license'] else None,
        'subscribers_count': get_additional_metric(repo_data['subscribers_url']),
        'contributors_count': get_additional_metric(repo_data['contributors_url']),
        'commits_count': get_commit_count(repo_data['commits_url'].split('{')[0]),
        'readme_size': get_readme_size(repo_data['full_name'])
    }
    return features

def get_additional_metric(url):
    try:
        response = requests.get(url, headers=HEADERS)
        if response.status_code == 200:
            return len(response.json())
    except:
        pass
    return 0

def get_commit_count(url):
    try:
        response = requests.get(url, headers=HEADERS)
        if response.status_code == 200:
            return len(response.json())
    except:
        pass
    return 0

def get_readme_size(repo_full_name):
    try:
        url = f'https://api.github.com/repos/{repo_full_name}/readme'
        response = requests.get(url, headers=HEADERS)
        if response.status_code == 200:
            return response.json().get('size', 0)
    except:
        pass
    return 0

# Process all repositories
c=1
features_list = []
for repo in top_repos:
    features = extract_features(repo)
    features_list.append(features)
    print(c)
    c+=1
    print(features)

features_df = pd.DataFrame(features_list)
features_df.to_csv('github_repo_features.csv', index=False)

1
{'name': 'freeCodeCamp', 'full_name': 'freeCodeCamp/freeCodeCamp', 'stars': 417516, 'forks': 39997, 'watchers': 417516, 'open_issues': 283, 'size': 497356, 'has_wiki': 0, 'has_projects': 1, 'has_downloads': 1, 'is_fork': 0, 'archived': 0, 'created_at': '2014-12-24T17:49:19Z', 'updated_at': '2025-05-04T07:38:21Z', 'pushed_at': '2025-05-03T11:08:48Z', 'language': 'TypeScript', 'license': 'bsd-3-clause', 'subscribers_count': 30, 'contributors_count': 30, 'commits_count': 30, 'readme_size': 32109}
2
{'name': 'build-your-own-x', 'full_name': 'codecrafters-io/build-your-own-x', 'stars': 377651, 'forks': 35155, 'watchers': 377651, 'open_issues': 373, 'size': 1061, 'has_wiki': 0, 'has_projects': 0, 'has_downloads': 1, 'is_fork': 0, 'archived': 0, 'created_at': '2018-05-09T12:03:18Z', 'updated_at': '2025-05-04T07:38:32Z', 'pushed_at': '2025-04-11T19:13:28Z', 'language': 'Markdown', 'license': None, 'subscribers_count': 30, 'contributors_count': 30, 'commits_count': 30, 'readme_size': 45203}
3

18
{'name': 'tensorflow', 'full_name': 'tensorflow/tensorflow', 'stars': 189761, 'forks': 74656, 'watchers': 189761, 'open_issues': 8698, 'size': 1186059, 'has_wiki': 0, 'has_projects': 1, 'has_downloads': 1, 'is_fork': 0, 'archived': 0, 'created_at': '2015-11-07T01:19:20Z', 'updated_at': '2025-05-04T06:25:56Z', 'pushed_at': '2025-05-04T07:36:50Z', 'language': 'C++', 'license': 'apache-2.0', 'subscribers_count': 30, 'contributors_count': 30, 'commits_count': 30, 'readme_size': 11912}
19
{'name': 'You-Dont-Know-JS', 'full_name': 'getify/You-Dont-Know-JS', 'stars': 181956, 'forks': 33617, 'watchers': 181956, 'open_issues': 125, 'size': 15059, 'has_wiki': 0, 'has_projects': 1, 'has_downloads': 1, 'is_fork': 0, 'archived': 0, 'created_at': '2013-11-16T02:37:24Z', 'updated_at': '2025-05-04T07:04:07Z', 'pushed_at': '2025-05-01T02:05:28Z', 'language': None, 'license': 'other', 'subscribers_count': 30, 'contributors_count': 30, 'commits_count': 30, 'readme_size': 4105}
20
{'name': 'CS-Notes', 

35
{'name': 'awesome-go', 'full_name': 'avelino/awesome-go', 'stars': 142973, 'forks': 12264, 'watchers': 142973, 'open_issues': 118, 'size': 10507, 'has_wiki': 0, 'has_projects': 0, 'has_downloads': 1, 'is_fork': 0, 'archived': 0, 'created_at': '2014-07-06T13:42:15Z', 'updated_at': '2025-05-04T06:54:48Z', 'pushed_at': '2025-05-01T04:45:23Z', 'language': 'Go', 'license': 'mit', 'subscribers_count': 30, 'contributors_count': 30, 'commits_count': 30, 'readme_size': 357854}
36
{'name': 'ollama', 'full_name': 'ollama/ollama', 'stars': 139431, 'forks': 11636, 'watchers': 139431, 'open_issues': 1783, 'size': 43685, 'has_wiki': 0, 'has_projects': 0, 'has_downloads': 1, 'is_fork': 0, 'archived': 0, 'created_at': '2023-06-26T19:39:32Z', 'updated_at': '2025-05-04T07:38:56Z', 'pushed_at': '2025-05-03T21:07:01Z', 'language': 'Go', 'license': 'mit', 'subscribers_count': 30, 'contributors_count': 30, 'commits_count': 30, 'readme_size': 37003}
37
{'name': 'youtube-dl', 'full_name': 'ytdl-org/youtube-

52
{'name': 'node', 'full_name': 'nodejs/node', 'stars': 110952, 'forks': 31443, 'watchers': 110952, 'open_issues': 2194, 'size': 1331213, 'has_wiki': 0, 'has_projects': 1, 'has_downloads': 1, 'is_fork': 0, 'archived': 0, 'created_at': '2014-11-26T19:57:11Z', 'updated_at': '2025-05-04T07:31:02Z', 'pushed_at': '2025-05-04T00:40:17Z', 'language': 'JavaScript', 'license': 'other', 'subscribers_count': 30, 'contributors_count': 30, 'commits_count': 30, 'readme_size': 40253}
53
{'name': 'yt-dlp', 'full_name': 'yt-dlp/yt-dlp', 'stars': 110577, 'forks': 8680, 'watchers': 110577, 'open_issues': 1901, 'size': 53949, 'has_wiki': 1, 'has_projects': 1, 'has_downloads': 1, 'is_fork': 0, 'archived': 0, 'created_at': '2020-10-26T04:22:55Z', 'updated_at': '2025-05-04T07:39:42Z', 'pushed_at': '2025-05-03T20:11:40Z', 'language': 'Python', 'license': 'unlicense', 'subscribers_count': 30, 'contributors_count': 30, 'commits_count': 30, 'readme_size': 168694}
54
{'name': 'd3', 'full_name': 'd3/d3', 'stars':

70
{'name': 'material-ui', 'full_name': 'mui/material-ui', 'stars': 95514, 'forks': 32556, 'watchers': 95514, 'open_issues': 1802, 'size': 687863, 'has_wiki': 0, 'has_projects': 1, 'has_downloads': 1, 'is_fork': 0, 'archived': 0, 'created_at': '2014-08-18T19:11:54Z', 'updated_at': '2025-05-04T06:52:01Z', 'pushed_at': '2025-05-04T03:23:57Z', 'language': 'TypeScript', 'license': 'mit', 'subscribers_count': 30, 'contributors_count': 30, 'commits_count': 30, 'readme_size': 10828}
71
{'name': 'free-for-dev', 'full_name': 'ripienaar/free-for-dev', 'stars': 95257, 'forks': 10302, 'watchers': 95257, 'open_issues': 12, 'size': 8572, 'has_wiki': 0, 'has_projects': 0, 'has_downloads': 1, 'is_fork': 0, 'archived': 0, 'created_at': '2015-03-18T21:06:26Z', 'updated_at': '2025-05-04T07:36:08Z', 'pushed_at': '2025-05-02T11:41:07Z', 'language': 'HTML', 'license': None, 'subscribers_count': 30, 'contributors_count': 30, 'commits_count': 30, 'readme_size': 297130}
72
{'name': 'ant-design', 'full_name': '

...

919
{'name': 'waifu2x', 'full_name': 'nagadomi/waifu2x', 'stars': 27844, 'forks': 2704, 'watchers': 27844, 'open_issues': 155, 'size': 709513, 'has_wiki': 1, 'has_projects': 0, 'has_downloads': 1, 'is_fork': 0, 'archived': 0, 'created_at': '2015-05-17T07:29:15Z', 'updated_at': '2025-05-04T04:51:35Z', 'pushed_at': '2023-05-04T00:50:48Z', 'language': 'Lua', 'license': 'mit', 'subscribers_count': 30, 'contributors_count': 30, 'commits_count': 30, 'readme_size': 8273}
920
{'name': 'Charts', 'full_name': 'ChartsOrg/Charts', 'stars': 27838, 'forks': 6029, 'watchers': 27838, 'open_issues': 996, 'size': 192506, 'has_wiki': 1, 'has_projects': 1, 'has_downloads': 1, 'is_fork': 0, 'archived': 0, 'created_at': '2015-03-20T10:49:12Z', 'updated_at': '2025-05-03T20:12:34Z', 'pushed_at': '2024-08-19T06:17:16Z', 'language': 'Swift', 'license': 'apache-2.0', 'subscribers_count': 30, 'contributors_count': 30, 'commits_count': 30, 'readme_size': 13296}
921
{'name': 'helm', 'full_name': 'helm/helm', 'stars

937
{'name': 'Daily-Interview-Question', 'full_name': 'Advanced-Frontend/Daily-Interview-Question', 'stars': 27490, 'forks': 3283, 'watchers': 27490, 'open_issues': 264, 'size': 199, 'has_wiki': 1, 'has_projects': 1, 'has_downloads': 1, 'is_fork': 0, 'archived': 0, 'created_at': '2019-01-19T10:49:00Z', 'updated_at': '2025-05-01T20:31:30Z', 'pushed_at': '2020-11-09T01:07:00Z', 'language': 'JavaScript', 'license': None, 'subscribers_count': 30, 'contributors_count': 6, 'commits_count': 30, 'readme_size': 8522}
938
{'name': 'tdesktop', 'full_name': 'telegramdesktop/tdesktop', 'stars': 27475, 'forks': 5512, 'watchers': 27475, 'open_issues': 893, 'size': 245310, 'has_wiki': 1, 'has_projects': 0, 'has_downloads': 1, 'is_fork': 0, 'archived': 0, 'created_at': '2014-05-02T12:36:31Z', 'updated_at': '2025-05-04T05:15:31Z', 'pushed_at': '2025-05-03T11:46:46Z', 'language': 'C++', 'license': 'other', 'subscribers_count': 30, 'contributors_count': 30, 'commits_count': 30, 'readme_size': 5142}
939
{'

954
{'name': 'mongoose', 'full_name': 'Automattic/mongoose', 'stars': 27214, 'forks': 3861, 'watchers': 27214, 'open_issues': 227, 'size': 138392, 'has_wiki': 1, 'has_projects': 1, 'has_downloads': 1, 'is_fork': 0, 'archived': 0, 'created_at': '2010-04-06T21:39:05Z', 'updated_at': '2025-05-03T19:23:58Z', 'pushed_at': '2025-05-02T18:55:55Z', 'language': 'JavaScript', 'license': 'mit', 'subscribers_count': 30, 'contributors_count': 30, 'commits_count': 30, 'readme_size': 13519}
955
{'name': 'vscodium', 'full_name': 'VSCodium/vscodium', 'stars': 27209, 'forks': 1266, 'watchers': 27209, 'open_issues': 132, 'size': 32671, 'has_wiki': 0, 'has_projects': 0, 'has_downloads': 1, 'is_fork': 0, 'archived': 0, 'created_at': '2018-08-13T14:26:58Z', 'updated_at': '2025-05-04T05:43:17Z', 'pushed_at': '2025-05-03T09:48:08Z', 'language': 'Shell', 'license': 'mit', 'subscribers_count': 30, 'contributors_count': 30, 'commits_count': 30, 'readme_size': 12542}
956
{'name': 'paperless-ngx', 'full_name': 'pa

972
{'name': 'angular-cli', 'full_name': 'angular/angular-cli', 'stars': 26903, 'forks': 11975, 'watchers': 26903, 'open_issues': 281, 'size': 189319, 'has_wiki': 0, 'has_projects': 0, 'has_downloads': 1, 'is_fork': 0, 'archived': 0, 'created_at': '2015-06-04T19:49:37Z', 'updated_at': '2025-05-04T05:56:35Z', 'pushed_at': '2025-05-03T16:05:09Z', 'language': 'TypeScript', 'license': 'mit', 'subscribers_count': 30, 'contributors_count': 30, 'commits_count': 30, 'readme_size': 10392}
973
{'name': 'Rectangle', 'full_name': 'rxhanson/Rectangle', 'stars': 26887, 'forks': 820, 'watchers': 26887, 'open_issues': 98, 'size': 5194, 'has_wiki': 1, 'has_projects': 1, 'has_downloads': 1, 'is_fork': 0, 'archived': 0, 'created_at': '2019-06-21T21:34:53Z', 'updated_at': '2025-05-04T06:58:22Z', 'pushed_at': '2025-04-05T04:43:54Z', 'language': 'Swift', 'license': 'other', 'subscribers_count': 30, 'contributors_count': 30, 'commits_count': 30, 'readme_size': 10944}
974
{'name': 'xgboost', 'full_name': 'dml

990
{'name': 'podman', 'full_name': 'containers/podman', 'stars': 26596, 'forks': 2612, 'watchers': 26596, 'open_issues': 816, 'size': 159780, 'has_wiki': 1, 'has_projects': 1, 'has_downloads': 1, 'is_fork': 0, 'archived': 0, 'created_at': '2017-11-01T15:01:27Z', 'updated_at': '2025-05-04T06:53:22Z', 'pushed_at': '2025-05-03T03:22:52Z', 'language': 'Go', 'license': 'apache-2.0', 'subscribers_count': 30, 'contributors_count': 30, 'commits_count': 30, 'readme_size': 12318}
991
{'name': 'supervision', 'full_name': 'roboflow/supervision', 'stars': 26552, 'forks': 2008, 'watchers': 26552, 'open_issues': 116, 'size': 2186425, 'has_wiki': 1, 'has_projects': 1, 'has_downloads': 1, 'is_fork': 0, 'archived': 0, 'created_at': '2022-11-28T14:08:44Z', 'updated_at': '2025-05-03T19:43:45Z', 'pushed_at': '2025-04-28T23:12:59Z', 'language': 'Python', 'license': 'mit', 'subscribers_count': 30, 'contributors_count': 30, 'commits_count': 30, 'readme_size': 11873}
992
{'name': 'openai-python', 'full_name':